# Введение в обработку текста на естественном языке

Материалы:

* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Разминка

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [ ]:
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий''' 

2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

## Лабораторная работа 9

### Расстояние редактирования

In [ ]:
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk
import random
from nltk.metrics import edit_distance
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from Levenshtein import distance as levenshtein_distance
from nltk.corpus import stopwords
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine
from itertools import combinations



1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [ ]:
# Скачиваем токенизатор
nltk.download('punkt')

df = pd.read_csv("preprocessed_descriptions.csv")

# Объединяем все описания в один текст
all_text = " ".join(df['description'].astype(str))

# Токенизация
tokens = word_tokenize(all_text)

# Множество уникальных слов
words = set(tokens)

print(f"Количество уникальных слов: {len(words)}")
print(sorted(list(words))[:20])  # первые 20 слов


[nltk_data] Downloading package punkt to C:\Users\khaiy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Количество уникальных слов: 29764
['!', '#', '$', '%', '&', "'", "''", "'00", "'03", "'04", "'05", "'06", "'07", "'08", "'09", "'10", "'1001", "'12", "'300", "'333"]


1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [ ]:
# Преобразуем во множество без знаков препинания
filtered_words = [w for w in words if w.isalpha()]

# Случайно выбираем 5 пар
sample_words = random.sample(filtered_words, 10)

# Группируем в пары
pairs = list(zip(sample_words[::2], sample_words[1::2]))

# Вычисляем расстояния
for w1, w2 in pairs:
    dist = edit_distance(w1, w2)
    print(f"{w1} — {w2} → расстояние редактирования: {dist}")


guestimate — closer → расстояние редактирования: 9
lunck — zea → расстояние редактирования: 5
volunteer — dedos → расстояние редактирования: 9
chlorophyll — saltscapes → расстояние редактирования: 10
cheesier — contessa → расстояние редактирования: 6


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [ ]:
def k_nearest_words(word, words, k):
    # Вычисляем расстояния Левенштейна от word до каждого слова из списка
    distances = [(levenshtein_distance(word, w), w) for w in words]
    
    # Находим k слов с наименьшими расстояниями
    closest = sorted(distances)[0:k]
    
    # Возвращаем только слова (без расстояний)
    return [w for _, w in closest]
k_nearest_words('flair', words, 5)


['flair', 'fair', 'affair', 'air', 'blais']

### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [15]:
# Скачиваем необходимые ресурсы
nltk.download('wordnet')
nltk.download('omw-1.4')

# Инициализируем инструменты
stemmer = SnowballStemmer("english")
lemmatizer = WordNetLemmatizer()

# Обработка слов
data = []
for word in words:
    stemmed = stemmer.stem(word)
    lemmatized = lemmatizer.lemmatize(word)
    data.append([word, stemmed, lemmatized])

# Создание DataFrame
df = pd.DataFrame(data, columns=["word", "stemmed_word", "normalized_word"])
df.set_index("word", inplace=True)

print(df)

[nltk_data] Downloading package wordnet to C:\Users\khaiy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\khaiy/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


          stemmed_word normalized_word
word                                  
imbergamo    imbergamo       imbergamo
vibrance       vibranc        vibrance
granita        granita         granita
tinkered        tinker        tinkered
'same-old     same-old       'same-old
...                ...             ...
sail              sail            sail
calories        calori         calorie
lakeland      lakeland        lakeland
smoothly        smooth        smoothly
faculty        faculti         faculty

[29764 rows x 2 columns]


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [54]:
# Скачиваем необходимые ресурсы
nltk.download('punkt')
nltk.download('stopwords')

# Объединяем все описания в одну строку и токенизируем
text = all_text
tokens = word_tokenize(text.lower())  

# Удаляем знаки пунктуации
tokens = [word for word in tokens if word.isalpha()]

stop_words = set(stopwords.words('english'))
total_words = len(tokens)

# Фильтруем токены — удаляем стоп-слова
tokens_filtered = [word for word in tokens if word not in stop_words]

# Считаем оставшиеся слова
filtered_words = len(tokens_filtered)

# Доля стоп-слов
stopword_ratio = (total_words - filtered_words) / total_words

# Частотный анализ
counter_before = Counter(tokens)
counter_after = Counter(tokens_filtered)

# Топ-10 слов до и после
top10_before = counter_before.most_common(10)
top10_after = counter_after.most_common(10)

print(f"Общее количество слов: {total_words}")
print(f"Осталось после удаления стоп-слов: {filtered_words}")
print(f"Доля стоп-слов: {stopword_ratio:.2%}")
print("\nТоп-10 слов ДО удаления стоп-слов:")
print(top10_before)
print("\nТоп-10 слов ПОСЛЕ удаления стоп-слов:")
print(top10_after)


[nltk_data] Downloading package punkt to C:\Users\khaiy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\khaiy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Общее количество слов: 1055765
Осталось после удаления стоп-слов: 555605
Доля стоп-слов: 47.37%

Топ-10 слов ДО удаления стоп-слов:
[('the', 40257), ('a', 35030), ('and', 30425), ('i', 27797), ('this', 27132), ('to', 23508), ('it', 23212), ('is', 20501), ('of', 18379), ('for', 15996)]

Топ-10 слов ПОСЛЕ удаления стоп-слов:
[('recipe', 15122), ('make', 6367), ('time', 5198), ('use', 4645), ('great', 4473), ('easy', 4206), ('like', 4186), ('one', 3916), ('good', 3847), ('made', 3819)]


### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [ ]:
df = pd.read_csv("preprocessed_descriptions.csv")


sample_descriptions = df["description"].sample(5)

vectorizer = TfidfVectorizer()
vector = vectorizer.fit_transform(sample_descriptions)

# Преобразуем результат в DataFrame (опционально, для наглядности)
vector_df = pd.DataFrame(vector.toarray())

# Добавим описание рецепта для сопоставления
vector_df["original_description"] = sample_descriptions.values

# Показываем результат
print(vector_df.head())


          0         1         2         3         4        5         6  \
0  0.000000  0.000000  0.000000  0.000000  0.000000  0.00000  0.000000   
1  0.150983  0.000000  0.101115  0.000000  0.000000  0.00000  0.000000   
2  0.000000  0.139922  0.093707  0.000000  0.000000  0.00000  0.139922   
3  0.000000  0.000000  0.285009  0.106393  0.106393  0.00000  0.000000   
4  0.000000  0.000000  0.000000  0.000000  0.000000  0.57735  0.000000   

          7         8         9  ...       119       120       121       122  \
0  0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000  0.169714   
1  0.150983  0.000000  0.150983  ...  0.000000  0.000000  0.000000  0.000000   
2  0.000000  0.000000  0.000000  ...  0.000000  0.279843  0.000000  0.000000   
3  0.000000  0.106393  0.000000  ...  0.106393  0.000000  0.106393  0.000000   
4  0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000  0.000000   

        123       124       125       126       127  \
0  0.000000  0.0000

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [ ]:
# Векторизация
vectorizer = TfidfVectorizer()
# descriptions = list(recipes.values())
vector = vectorizer.fit_transform(vector_df["original_description"])

# Имена рецептов — используем индексы или сгенерированные имена
names = [f"Recipe {i}" for i in vector_df.index]

# Создаём DataFrame для хранения расстояний
n = len(names)
dist_matrix = pd.DataFrame(index=names, columns=names, dtype=float)

# Заполняем матрицу косинусных расстояний
for i in range(n):
    for j in range(n):
        vec_i = vector[i].toarray()[0]
        vec_j = vector[j].toarray()[0]
        dist = cosine(vec_i, vec_j)
        dist_matrix.iloc[i, j] = dist

print(dist_matrix.round(3))

          Recipe 0  Recipe 1  Recipe 2  Recipe 3  Recipe 4
Recipe 0     0.000     0.922     0.992     0.958       1.0
Recipe 1     0.922     0.000     0.939     0.829       1.0
Recipe 2     0.992     0.939     0.000     0.806       1.0
Recipe 3     0.958     0.829     0.806     0.000       1.0
Recipe 4     1.000     1.000     1.000     1.000       0.0


3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).

In [ ]:
# Найдём индексы минимального ненулевого расстояния
min_dist = np.inf
closest_pair = (None, None)

for i in range(len(names)):
    for j in range(i + 1, len(names)):
        dist = dist_matrix.iloc[i, j]
        if dist < min_dist:
            min_dist = dist
            closest_pair = (names[i], names[j])

print(f"Наиболее похожие рецепты: {closest_pair[0]} и {closest_pair[1]}")
print(f"Косинусное расстояние между ними: {min_dist:.3f}")

# Косинусное расстояние между их описаниями составляет 0.806, что говорит о невысокой степени текстового сходства.


Наиболее похожие рецепты: we hosted hamburger barbeques  и so it's another night of tryin
Косинусное расстояние между ними: 0.806
